# Preparation

In [1]:
import torch
torch.__version__

'2.3.0a0+6ddf5cf85e.nv24.04'

In [2]:
import os
import pandas as pd
import numpy as np
import random
from torch import nn

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from model import *
from dataset import *
from train import *

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
BATCH_SIZE = 256
NUMEPOCHS = 50

In [5]:
# import pandas as pd
# df = pd.read_csv("Data/ml-1m.train.rating",sep='\t', header=None, names=['uid', 'mid', 'rating', 'timestamp'], engine='python')
# df

In [6]:
# df2 = pd.read_csv("Data/ml-1m.test.negative",sep = "\t",header=None)
# df2

# Trian and Test

### Compare the three methods 

In [7]:
model_config_mf = {
    "model_mark": "gmf(mf_dim=8)",
    'embedding_dim_mf': 8,
    # "mlp_layers(X)" : 0,
    # 'mlp_layers': [ 32, 16, 8],
    'model_type': 'GMF'     #　MLP, NeuMF
}

model_config_mlp = {
    "model_mark": "mlp(mlp_layer=3)",
    "mlp_layers(X)" : 3,
    'mlp_layers': [32, 16, 8],
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_config_neumf = {
    "model_mark": "neumf(mf_dim=8,mlp_layer=3)",
    'embedding_dim_mf': 8,
    "mlp_layers(X)" : 3,
    'mlp_layers': [32, 16, 8],
    'model_type': 'NeuMF'     #　MLP, NeuMF
}

model_train(model_config_mf, seed = 42, num_of_negatives=4, num_of_epochs=30)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'model_mark': 'gmf(mf_dim=8)', 'embedding_dim_mf': 8, 'model_type': 'GMF', 'num_users': 6040, 'num_items': 3706}


  3%|▎         | 1/30 [00:57<27:36, 57.12s/it]

Epoch 1, Loss: 0.0027, HR@10: 0.1084, NDCG@10: 0.0498


  7%|▋         | 2/30 [02:00<28:21, 60.77s/it]

Epoch 2, Loss: 0.0020, HR@10: 0.4161, NDCG@10: 0.2293


...

Epoch 28, Loss: 0.0010, HR@10: 0.6298, NDCG@10: 0.3614


 97%|█████████▋| 29/30 [31:24<01:03, 63.45s/it]

Epoch 29, Loss: 0.0010, HR@10: 0.6303, NDCG@10: 0.3611


100%|██████████| 30/30 [32:49<00:00, 65.65s/it]

Epoch 30, Loss: 0.0010, HR@10: 0.6311, NDCG@10: 0.3611
Best HR: 0.6311258278145695, Best NDCG: 0.3622338343922371 at Epoch 26


In [8]:
model_train(model_config_mlp, seed = 42, num_of_negatives=4, num_of_epochs=30)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'model_mark': 'mlp(mlp_layer=3)', 'mlp_layers(X)': 3, 'mlp_layers': [32, 16, 8], 'model_type': 'MLP', 'num_users': 6040, 'num_items': 3706}


  3%|▎         | 1/30 [01:57<56:58, 117.90s/it]

Epoch 1, Loss: 0.0015, HR@10: 0.4434, NDCG@10: 0.2497


  7%|▋         | 2/30 [03:22<45:59, 98.56s/it] 

Epoch 2, Loss: 0.0014, HR@10: 0.4515, NDCG@10: 0.2518


...

Epoch 28, Loss: 0.0011, HR@10: 0.5644, NDCG@10: 0.3135


 97%|█████████▋| 29/30 [45:14<01:27, 87.99s/it]

Epoch 29, Loss: 0.0011, HR@10: 0.5581, NDCG@10: 0.3146


100%|██████████| 30/30 [46:49<00:00, 93.65s/it]

Epoch 30, Loss: 0.0011, HR@10: 0.5598, NDCG@10: 0.3126
Best HR: 0.5667218543046357, Best NDCG: 0.3158625751324939 at Epoch 24


In [9]:
model_train(model_config_neumf, seed = 42, num_of_negatives=4, num_of_epochs=30)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'model_mark': 'neumf(mf_dim=8,mlp_layer=3)', 'embedding_dim_mf': 8, 'mlp_layers(X)': 3, 'mlp_layers': [32, 16, 8], 'model_type': 'NeuMF', 'num_users': 6040, 'num_items': 3706}


  3%|▎         | 1/30 [01:34<45:50, 94.85s/it]

Epoch 1, Loss: 0.0015, HR@10: 0.4462, NDCG@10: 0.2497


  7%|▋         | 2/30 [03:20<47:20, 101.44s/it]

Epoch 2, Loss: 0.0014, HR@10: 0.4482, NDCG@10: 0.2512


...

Epoch 28, Loss: 0.0010, HR@10: 0.6417, NDCG@10: 0.3677


 97%|█████████▋| 29/30 [47:07<01:31, 91.52s/it]

Epoch 29, Loss: 0.0010, HR@10: 0.6349, NDCG@10: 0.3656


100%|██████████| 30/30 [48:49<00:00, 97.64s/it]

Epoch 30, Loss: 0.0010, HR@10: 0.6409, NDCG@10: 0.3672
Best HR: 0.6425496688741722, Best NDCG: 0.3660134329731621 at Epoch 17


for movielens

HR, NDCG

-   GMF(Factor-8, X-0): 0.6311, 0.3622
-   MLP(Factor-8, X-3): 0.5667, 0.3159
-   NeuMF(Factor-8, X-3): 0.6425, 0.3660

## Reproduce the ablation study --- MLP with different layers

 K: 8
 
 layers (X): 0, 1, 2, 3, 4

In [10]:
model_config_mlp0 = {
    "model_mark": "mlp(mlp_layer=0)",
    "mlp_layers(X)" : 0,
    'mlp_layers': [8],
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_config_mlp1 = {
    "model_mark": "mlp(mlp_layer=1)",
    "mlp_layers(X)" : 1,
    'mlp_layers': [8],
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_config_mlp2 = {
    "model_mark": "mlp(mlp_layer=2)",
    "mlp_layers(X)" : 2,
    'mlp_layers': [16, 8],
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_config_mlp3 = {
    "model_mark": "mlp(mlp_layer=3)",
    "mlp_layers(X)" : 4,
    'mlp_layers': [32, 16, 8],
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_config_mlp4 = {
    "model_mark": "mlp(mlp_layer=4)",
    "mlp_layers(X)" : 4,
    'mlp_layers': [64, 32, 16, 8],
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_train(model_config_mlp0, seed = 42, num_of_negatives=4, num_of_epochs=30)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'model_mark': 'mlp(mlp_layer=0)', 'mlp_layers(X)': 0, 'mlp_layers': [8], 'model_type': 'MLP', 'num_users': 6040, 'num_items': 3706}


  3%|▎         | 1/30 [00:50<24:25, 50.52s/it]

Epoch 1, Loss: 0.0016, HR@10: 0.4470, NDCG@10: 0.2490


  7%|▋         | 2/30 [01:44<24:25, 52.34s/it]

Epoch 2, Loss: 0.0014, HR@10: 0.4505, NDCG@10: 0.2497


...

Epoch 28, Loss: 0.0014, HR@10: 0.4497, NDCG@10: 0.2518


 97%|█████████▋| 29/30 [24:28<00:50, 50.18s/it]

Epoch 29, Loss: 0.0014, HR@10: 0.4500, NDCG@10: 0.2519


100%|██████████| 30/30 [25:22<00:00, 50.74s/it]

Epoch 30, Loss: 0.0014, HR@10: 0.4503, NDCG@10: 0.2515
Best HR: 0.45331125827814567, Best NDCG: 0.25205759944116024 at Epoch 12


In [11]:
model_train(model_config_mlp1, seed = 42, num_of_negatives=4, num_of_epochs=30)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'model_mark': 'mlp(mlp_layer=1)', 'mlp_layers(X)': 1, 'mlp_layers': [8], 'model_type': 'MLP', 'num_users': 6040, 'num_items': 3706}


  3%|▎         | 1/30 [00:48<23:18, 48.23s/it]

Epoch 1, Loss: 0.0016, HR@10: 0.4470, NDCG@10: 0.2490


  7%|▋         | 2/30 [01:42<24:04, 51.60s/it]

Epoch 2, Loss: 0.0014, HR@10: 0.4505, NDCG@10: 0.2497


...

Epoch 28, Loss: 0.0014, HR@10: 0.4497, NDCG@10: 0.2518


 97%|█████████▋| 29/30 [24:14<00:50, 50.41s/it]

Epoch 29, Loss: 0.0014, HR@10: 0.4500, NDCG@10: 0.2519


100%|██████████| 30/30 [25:06<00:00, 50.22s/it]

Epoch 30, Loss: 0.0014, HR@10: 0.4503, NDCG@10: 0.2515
Best HR: 0.45331125827814567, Best NDCG: 0.25205759944116024 at Epoch 12


In [12]:
model_train(model_config_mlp2, seed = 42, num_of_negatives=4, num_of_epochs=30)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'model_mark': 'mlp(mlp_layer=2)', 'mlp_layers(X)': 2, 'mlp_layers': [16, 8], 'model_type': 'MLP', 'num_users': 6040, 'num_items': 3706}


  3%|▎         | 1/30 [01:07<32:39, 67.56s/it]

Epoch 1, Loss: 0.0015, HR@10: 0.4523, NDCG@10: 0.2499


  7%|▋         | 2/30 [02:14<31:20, 67.17s/it]

Epoch 2, Loss: 0.0014, HR@10: 0.4508, NDCG@10: 0.2501


...

Epoch 28, Loss: 0.0013, HR@10: 0.5053, NDCG@10: 0.2826


 97%|█████████▋| 29/30 [31:03<00:56, 56.15s/it]

Epoch 29, Loss: 0.0013, HR@10: 0.5104, NDCG@10: 0.2847


100%|██████████| 30/30 [32:04<00:00, 64.17s/it]

Epoch 30, Loss: 0.0013, HR@10: 0.5103, NDCG@10: 0.2848
Best HR: 0.510430463576159, Best NDCG: 0.28466994624063796 at Epoch 28


In [13]:
model_train(model_config_mlp3, seed = 42, num_of_negatives=4, num_of_epochs=30)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'model_mark': 'mlp(mlp_layer=3)', 'mlp_layers(X)': 4, 'mlp_layers': [32, 16, 8], 'model_type': 'MLP', 'num_users': 6040, 'num_items': 3706}


  3%|▎         | 1/30 [00:35<17:11, 35.56s/it]

Epoch 1, Loss: 0.0015, HR@10: 0.4434, NDCG@10: 0.2497


  7%|▋         | 2/30 [01:38<24:04, 51.58s/it]

Epoch 2, Loss: 0.0014, HR@10: 0.4515, NDCG@10: 0.2518


...

Epoch 28, Loss: 0.0011, HR@10: 0.5644, NDCG@10: 0.3135


 97%|█████████▋| 29/30 [38:34<01:22, 82.10s/it]

Epoch 29, Loss: 0.0011, HR@10: 0.5581, NDCG@10: 0.3146


100%|██████████| 30/30 [39:58<00:00, 79.96s/it]

Epoch 30, Loss: 0.0011, HR@10: 0.5598, NDCG@10: 0.3126
Best HR: 0.5667218543046357, Best NDCG: 0.3158625751324939 at Epoch 24


In [14]:
model_train(model_config_mlp4, seed = 42, num_of_negatives=4, num_of_epochs=30)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'model_mark': 'mlp(mlp_layer=4)', 'mlp_layers(X)': 4, 'mlp_layers': [64, 32, 16, 8], 'model_type': 'MLP', 'num_users': 6040, 'num_items': 3706}


  3%|▎         | 1/30 [01:38<47:50, 98.99s/it]

Epoch 1, Loss: 0.0014, HR@10: 0.4417, NDCG@10: 0.2477


  7%|▋         | 2/30 [03:14<45:12, 96.88s/it]

Epoch 2, Loss: 0.0014, HR@10: 0.4543, NDCG@10: 0.2520


...

Epoch 28, Loss: 0.0010, HR@10: 0.6002, NDCG@10: 0.3411


 97%|█████████▋| 29/30 [47:57<01:39, 99.25s/it]

Epoch 29, Loss: 0.0010, HR@10: 0.5977, NDCG@10: 0.3404


100%|██████████| 30/30 [49:37<00:00, 99.23s/it]

Epoch 30, Loss: 0.0010, HR@10: 0.5982, NDCG@10: 0.3419
Best HR: 0.6052980132450331, Best NDCG: 0.3423428631718419 at Epoch 21


HR, NDCG

-   MLP0: 0.4533, 0.2521
-   MLP1: 0.4533, 0.2521
-   MLP2: 0.5104, 0.2847
-   MLP3: 0.5667, 0.3159
-   MLP4: 0.6053, 0.3423